<h1>Segmenting and Clustering Neighborhoods in Toronto - Part 1</h1>

In [2]:
!conda install -c conda-forge lxml --yes
import requests
import lxml.html as lh
import pandas as pd

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    libxslt-1.1.33             |       h7d1a2b0_0         426 KB
    lxml-3.8.0                 |           py36_0         3.8 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.5 MB

The following NEW packages will be INSTALLED:

  libxslt            pkgs/main/linux-64::libxslt-1.1.33-h7d1a2b0_0
  lxml               conda-forge/linux-64::lxml-3.8.0-py36_0

The following packages will be UPDATED:

  ca-certificates                     2020.4.5.2-hecda079_0 --> 

In [3]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [4]:
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [5]:
tr_elements = doc.xpath('//tr')

#Create empty list
col=[]
i=0

#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postal Code
"
2:"Borough
"
3:"Neighborhood
"


In [6]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [7]:
# Check the length of each column. Ideally, they should all be the same
[len(C) for (title,C) in col]

[181, 181, 181]

In [8]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.head()

,Postal Code\n,Borough\n,Neighborhood\n
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [17]:
df.columns = ['Postcode', 'Neighbourhood', 'Borough',]
cols = df.columns.tolist()
cols
cols = cols[-1:] + cols[:-1]

df = df[cols]
df.rename(columns={'Borough':'Postalcode', 'Postcode':'Borough'}, inplace='True')
df.head()

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [20]:
#Cleaning the messy string \n
'''df.columns = ['Postalcode', 'Borough', 'Neighbourhood']
df = df.replace('\n',' ', regex=True)
df.head()'''
d

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [32]:
#Dropping all cells with a borough that is Not assigned
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)

df = df.reset_index(drop=True)
df.head(10)

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae
6,M1J,Scarborough,Scarborough Village
7,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
8,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
9,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"


In [25]:
#Combining Neighbourhoods based on similar Postcode and Borough
df = df.groupby(['Postalcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.columns = ['Postalcode','Borough','Neighbourhood']
df.head(10)

,Postalcode,Borough,Neighbourhood
0,,Canadian postal codes,
1,M1A,Not assigned,Not assigned
2,M1B,Scarborough,"Malvern, Rouge"
3,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
4,M1E,Scarborough,"Guildwood, Morningside, West Hill"
5,M1G,Scarborough,Woburn
6,M1H,Scarborough,Cedarbrae
7,M1J,Scarborough,Scarborough Village
8,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
9,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"


In [26]:
#Removing any space in the start of the string
df['Neighbourhood'] = df['Neighbourhood'].str.strip()

#Assigning Borough values to the Neignbourhood where vlaue is "Not assigned"
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']

In [27]:
# Check the shape of the data frame
df.shape

(181, 3)

In [28]:
#Save this file to a csv
df.to_csv(r'df_can.csv')
print('File saved in csv format')

File saved in csv format
